In [40]:
import pandas as pd
import duckdb

In [41]:
root_path = ''
file_path = ''
table_name = ''
fabric_workspace = ''
sql_catalog_path = ''
lakehouse_name = ''

In [42]:
def data_transformation(options
                        ,root_path
                        ,file_path
                        ,sql_catalog_path
                        ,fabric_workspace
                        ,table_name, lakehouse_name
                        ):
    
    options_str = ', '.join(f'{key}={repr(value)}' for key, value in options.items())

    entity_path = f"{root_path}/{file_path}/*.jsonl"

    print(entity_path)

    sql_path = f"{root_path}/{sql_catalog_path}"

    try:
        with open(sql_path, "r", encoding="utf-8") as file:
            content = file.read()
            content = content.replace("__FILE_PATH__",entity_path).replace("__OPTIONS_STR__", options_str)
            print(f"""{content}""")
            df_output = duckdb.sql(f"""
                                    {content}
                                """).pl()

            (df_output).write_delta(f"abfss://{fabric_workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse/Tables/{table_name}",
                                mode="overwrite"
                       )
            print(f"Written to {table_name} successfully")
            #print(df_output)
    except FileNotFoundError:
        print(f"Error: The file '{sql_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [44]:
if __name__ == "__main__":
    options = {
                'compression': "gzip"
            }

    data_transformation(options=options
                        ,root_path=root_path
                        ,file_path=file_path
                        ,sql_catalog_path=sql_catalog_path
                        ,fabric_workspace=fabric_workspace
                        ,table_name=table_name
                        ,lakehouse_name = lakehouse_name
                        )

/lakehouse/default/Files//northwind/bronze/raw__northwind__orders/*.jsonl
SELECT order_id
       ,customer_id
       ,employee_id
       ,CAST(strftime(CAST(order_date AS DATE), '%Y/%m/%d') AS DATE) AS order_date
       ,CAST(strftime(CAST(required_date AS DATE), '%Y/%m/%d') AS DATE) AS required_date
       ,CAST(strftime(CAST(shipped_date AS DATE), '%Y/%m/%d') AS DATE) AS shipped_date
       ,ship_via
       ,freight
       ,ship_name
       ,ship_address
       ,ship_city
       ,ship_postal_code
       ,ship_country
       ,COALESCE(ship_region, 'unknown') AS ship_region
FROM read_json('/lakehouse/default/Files//northwind/bronze/raw__northwind__orders/*.jsonl', compression='gzip')
Written to clean__northwind__orders successfully
